In [18]:
!pip install ultralytics
!pip install "git+https://github.com/jacobgil/pytorch-grad-cam.git"

  Cloning https://github.com/jacobgil/pytorch-grad-cam.git to /tmp/pip-req-build-ow0l60id
  Running command git clone --filter=blob:none --quiet https://github.com/jacobgil/pytorch-grad-cam.git /tmp/pip-req-build-ow0l60id
  Resolved https://github.com/jacobgil/pytorch-grad-cam.git to commit 781dbc0d16ffa95b6d18b96b7b829840a82d93d1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [19]:
import torch
from pytorch_grad_cam import GradCAM, XGradCAM, EigenCAM

print("Grad-CAM available ✅", GradCAM, XGradCAM, EigenCAM)


Grad-CAM available ✅ <class 'pytorch_grad_cam.grad_cam.GradCAM'> <class 'pytorch_grad_cam.xgrad_cam.XGradCAM'> <class 'pytorch_grad_cam.eigen_cam.EigenCAM'>


In [20]:
%%writefile /content/yolov11_heatmap.py
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import sys, os, shutil
import torch, cv2
import numpy as np
from tqdm import trange
from PIL import Image

# (Optional) If you had yolov11 repo on disk; harmless if unused
if '/content/yolov11' not in sys.path:
    sys.path.append('/content/yolov11')

# Ultralytics utils
from ultralytics.utils.ops import non_max_suppression, xywh2xyxy
from ultralytics import YOLO

# Grad-CAM family
from pytorch_grad_cam import GradCAMPlusPlus, GradCAM, XGradCAM, EigenCAM, HiResCAM, LayerCAM, RandomCAM, EigenGradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image, scale_cam_image

np.random.seed(0)

class ActivationsAndGradients:
    def __init__(self, model, target_layers, reshape_transform):
        self.model = model
        self.gradients = []
        self.activations = []
        self.reshape_transform = reshape_transform
        self.handles = []
        for tl in target_layers:
            self.handles.append(tl.register_forward_hook(self._save_activation))
            self.handles.append(tl.register_forward_hook(self._save_gradient))

    def _save_activation(self, module, inp, out):
        act = out
        if self.reshape_transform is not None:
            act = self.reshape_transform(act)
        self.activations.append(act.detach().cpu())

    def _save_gradient(self, module, inp, out):
        if not hasattr(out, "requires_grad") or not out.requires_grad:
            return
        def _store(g):
            if self.reshape_transform is not None:
                g = self.reshape_transform(g)
            self.gradients = [g.detach().cpu()] + self.gradients
        out.register_hook(_store)

    def post_process(self, result):
        # result expected: [B, N, 4+nc] in xywh format (pre-NMS)
        logits_ = result[:, 4:]
        boxes_  = result[:, :4]
        _, indices = torch.sort(logits_.max(1)[0], descending=True)
        return (
            torch.transpose(logits_[0], 0, 1)[indices[0]],
            torch.transpose(boxes_[0],  0, 1)[indices[0]],
            xywh2xyxy(torch.transpose(boxes_[0], 0, 1)[indices[0]]).detach().cpu().numpy(),
        )

    def __call__(self, x):
        self.gradients = []
        self.activations = []
        model_output = self.model(x)  # raw forward on nn.Module
        if isinstance(model_output, (list, tuple)):
            model_output = model_output[0]
        post_logits, pre_post_boxes, _ = self.post_process(model_output)
        return [[post_logits, pre_post_boxes]]

    def release(self):
        for h in self.handles:
            h.remove()

class Yolov11Target(torch.nn.Module):
    def __init__(self, output_type: str, conf: float, ratio: float) -> None:
        super().__init__()
        self.output_type = output_type  # 'class' | 'box' | 'all'
        self.conf = conf
        self.ratio = ratio

    def forward(self, data):
        post_logits, pre_post_boxes = data
        result_terms = []
        top_k = max(1, int(post_logits.size(0) * self.ratio))
        for i in trange(top_k, leave=False, desc="Backprop targets"):
            best_score = float(post_logits[i].max())
            if best_score < self.conf:
                break
            if self.output_type in ('class', 'all'):
                result_terms.append(post_logits[i].max())
            if self.output_type in ('box', 'all'):
                for j in range(4):
                    result_terms.append(pre_post_boxes[i, j])
        if not result_terms:
            return torch.tensor(0.0, device=pre_post_boxes.device)
        return sum(result_terms)

class Yolov11Heatmap:
    def __init__(self, weight, device, method, layer, backward_type, conf_threshold, ratio, show_box, renormalize, img_size=640):
        self.device = torch.device(device)
        self.img_size = img_size

        # ✅ Load Ultralytics YOLO (v8/11) — no torch.load / attempt_load
        self.yolo = YOLO(weight)                 # e.g. "runs/detect/train/weights/best.pt"
        self.model = self.yolo.model.to(self.device).eval()  # nn.Module

        # ✅ Class names from either place
        self.model_names = getattr(self.yolo, "names", None) or getattr(self.model, "names", None)
        if self.model_names is None:
            raise RuntimeError("Could not find class names on loaded model.")

        for p in self.model.parameters():
            p.requires_grad_(True)

        self.target = Yolov11Target(backward_type, conf_threshold, ratio)

        # pick feature layers by index from model.model (nn.Sequential-like)
        self.target_layers = [self.model.model[l] for l in layer]
        method_cls = eval(method)
        # Newer pytorch-grad-cam builds don't take `use_cuda`; device handled internally
        self.method = method_cls(self.model, self.target_layers)
        # Override with our custom hooks so post_process works as expected
        self.method.activations_and_grads = ActivationsAndGradients(self.model, self.target_layers, None)

        self.conf_threshold = conf_threshold
        self.show_box = show_box
        self.renormalize = renormalize
        self.colors = np.random.uniform(0, 255, size=(len(self.model_names), 3)).astype(np.int32)

    def _preprocess(self, bgr):
        # Simple square resize + normalize (YOLO11-friendly, avoids yolov11 letterbox)
        rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
        rgb = cv2.resize(rgb, (self.img_size, self.img_size), interpolation=cv2.INTER_LINEAR)
        imgf = rgb.astype(np.float32) / 255.0
        tensor = torch.from_numpy(np.transpose(imgf, (2, 0, 1))).unsqueeze(0).to(self.device)
        return imgf, tensor

    def _post_process(self, pred):
        """
        pred: raw model output for a batch; support a few shapes:
        - Tensor [B, N, 4+nc]
        - List/tuple with first element as above
        Returns: det tensor from Ultralytics NMS (xyxy, conf, cls) for batch 0
        """
        if isinstance(pred, (list, tuple)):
            pred = pred[0]
        if pred.dim() == 3:
            pred_b0 = pred[0]
        elif pred.dim() == 2:
            pred_b0 = pred
        else:
            raise RuntimeError(f"Unexpected prediction shape: {tuple(pred.shape)}")

        # Build [B, N, 5+nc] as expected by non_max_suppression:
        # [x, y, w, h, obj, cls_probs...]; synthesize obj as max class prob
        class_probs = pred_b0[:, 4:]
        obj_like = class_probs.max(dim=1, keepdim=True).values
        aligned = torch.cat([pred_b0[:, :4], obj_like, class_probs], dim=1).unsqueeze(0)

        det = non_max_suppression(aligned, conf_thres=self.conf_threshold, iou_thres=0.65)[0]
        return det

    def _draw_detections(self, box, color, name, img):
        x1, y1, x2, y2 = list(map(int, list(box)))
        x1, y1 = max(x1, 0), max(y1, 0)
        x2, y2 = max(x2, 0), max(y2, 0)
        cv2.rectangle(img, (x1, y1), (x2, y2), tuple(int(x) for x in color), 2)
        cv2.putText(img, str(name), (x1, max(0, y1 - 6)), cv2.FONT_HERSHEY_SIMPLEX,
                    0.8, tuple(int(x) for x in color), 2, lineType=cv2.LINE_AA)
        return img

    def _renormalize_cam_in_bboxes(self, boxes, image_float_np, grayscale_cam):
        H, W = grayscale_cam.shape
        renorm = np.zeros_like(grayscale_cam, dtype=np.float32)
        for x1, y1, x2, y2 in boxes:
            x1, y1 = max(int(x1), 0), max(int(y1), 0)
            x2, y2 = min(int(x2), W - 1), min(int(y2), H - 1)
            if x2 > x1 and y2 > y1:
                renorm[y1:y2, x1:x2] = scale_cam_image(grayscale_cam[y1:y2, x1:x2].copy())
        renorm = scale_cam_image(renorm)
        return show_cam_on_image(image_float_np, renorm, use_rgb=True)

    def _process_one(self, img_path, save_file):
        bgr = cv2.imread(img_path)
        if bgr is None:
            raise FileNotFoundError(f"Could not read image: {img_path}")
        imgf, tensor = self._preprocess(bgr)

        try:
            grayscale_cam = self.method(tensor, [self.target])
        except AttributeError:
            return
        grayscale_cam = grayscale_cam[0, :]
        cam_image = show_cam_on_image(imgf, grayscale_cam, use_rgb=True)

        with torch.no_grad():
            raw_pred = self.model(tensor)
            det = self._post_process(raw_pred)

        if det is not None and len(det):
            if self.renormalize:
                cam_image = self._renormalize_cam_in_bboxes(
                    det[:, :4].detach().cpu().numpy().astype(np.int32),
                    imgf, grayscale_cam
                )
            if self.show_box:
                for d in det:
                    d = d.detach().cpu().numpy()
                    cls_id = int(d[5])
                    conf = float(d[4])
                    label = f'{self.model_names[cls_id]} {conf:.2f}'
                    cam_image = self._draw_detections(d[:4], self.colors[cls_id], label, cam_image)

        Image.fromarray(cam_image).save(save_file)

    def __call__(self, img_path, save_dir):
        if os.path.exists(save_dir):
            if os.path.isdir(save_dir):
                shutil.rmtree(save_dir)
            else:
                os.remove(save_dir)
        os.makedirs(save_dir, exist_ok=True)

        if os.path.isdir(img_path):
            for fname in os.listdir(img_path):
                in_path = os.path.join(img_path, fname)
                if os.path.isdir(in_path):
                    continue
                out_path = os.path.join(save_dir, fname)
                self._process_one(in_path, out_path)
        else:
            self._process_one(img_path, os.path.join(save_dir, 'result.png'))

def get_params():
    return {
        'weight': '/content/best.pt',  # <-- point to your Ultralytics .pt
        'device': 'cuda:0' if torch.cuda.is_available() else 'cpu',
        'method': 'EigenCAM',          # This will be overridden
        'layer': [12, 15, 18],         # adjust to valid indices for your model
        'backward_type': 'all',
        'conf_threshold': 0.20,
        'ratio': 0.05,
        'show_box': True,
        'renormalize': False,
        'img_size': 640
    }

if __name__ == '__main__':
    # Define all methods to test
    methods = [
        'GradCAMPlusPlus',
        'GradCAM',
        'XGradCAM',
        'EigenCAM',
        'HiResCAM',
        'LayerCAM',
        'RandomCAM',
        'EigenGradCAM'
    ]

    img_path = '/content/test_imgs'
    base_save_dir = '/content/result_heatmaps'

    # Run each method
    for method in methods:
        print(f'Running {method}...')
        params = get_params()
        params['method'] = method
        runner = Yolov11Heatmap(**params)
        method_save_dir = os.path.join(base_save_dir, method)
        runner(img_path, method_save_dir)
        print(f'✅ {method} completed. Results saved to {method_save_dir}')

    print('✅ All methods completed. Check /content/result_heatmaps for results')

Overwriting /content/yolov11_heatmap.py


In [21]:

!python /content/yolov11_heatmap.py


Running GradCAMPlusPlus...
✅ GradCAMPlusPlus completed. Results saved to /content/result_heatmaps/GradCAMPlusPlus
Running GradCAM...
✅ GradCAM completed. Results saved to /content/result_heatmaps/GradCAM
Running XGradCAM...
✅ XGradCAM completed. Results saved to /content/result_heatmaps/XGradCAM
Running EigenCAM...
✅ EigenCAM completed. Results saved to /content/result_heatmaps/EigenCAM
Running HiResCAM...
✅ HiResCAM completed. Results saved to /content/result_heatmaps/HiResCAM
Running LayerCAM...
✅ LayerCAM completed. Results saved to /content/result_heatmaps/LayerCAM
Running RandomCAM...
✅ RandomCAM completed. Results saved to /content/result_heatmaps/RandomCAM
Running EigenGradCAM...
✅ EigenGradCAM completed. Results saved to /content/result_heatmaps/EigenGradCAM
✅ All methods completed. Check /content/result_heatmaps for results
